In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
os.chdir('..')
sys.path.append('src')

In [ ]:
from rasterio.features import rasterize

In [ ]:
import rasterio
import torch
import numpy as np
from pathlib import Path
import pandas as pd
import torchvision
from dataclasses import dataclass
from omegaconf import OmegaConf
from tqdm import tqdm
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib widget 

from data import read_ann, convert_ann

In [ ]:
def start_points(size, split_size, overlap=0):
    points = [0]
    stride = int(split_size * (1-overlap))
    counter = 1
    while True:
        pt = stride * counter
        if pt + split_size >= size:
            points.append(size - split_size)
            break
        else:
            points.append(pt)
        counter += 1
    return points



def splitter(img, crop_w, crop_h):    
    img_h, img_w, *_ = img.shape
    X_points = start_points(img_w, crop_w, 0.5)
    Y_points = start_points(img_h, crop_h, 0.5)
    
    for i in Y_points:
        for j in X_points:
            split = img[i:i+crop_h, j:j+crop_w]
            #cv2.imwrite('{}_{}.{}'.format(name, count, frmt), split)
            yield split, j, i, crop_h, crop_w

In [ ]:
df = pd.read_csv('input/hmib/train.csv')

In [ ]:
imgs_path = Path('input/hmib/train_images/')
ann_path = Path('input/hmib/train_annotations/')
dst = Path('input/preprocessed/test')
dst.mkdir()
(dst / 'images').mkdir()
(dst / 'masks').mkdir()

In [ ]:
#tdf = df[df.organ == 'spleen']
tdf = df

In [ ]:
H, W = 1024, 1024
ch, cw = 512,512
imgs = []
masks = []
for i,row in tqdm(tdf.iterrows(), total=len(tdf)):
    #if i < 10:
    #    continue
    #break
    f = imgs_path / str(row['id'])
    f = f.with_suffix('.tiff')
    fd = rasterio.open(f)
    a = fd.read().transpose(1,2,0)
    fd.close()
    h,w,c = a.shape
    ar = cv2.resize(a, (W, H))
    ar = ar.astype(np.uint8)
    name = f.with_suffix('.png').name
    
    ss = splitter(ar, ch, cw)
    name, ext = name.split('.')
    for ii, (s,*_) in enumerate(ss):
        fixname = f'{name}_{ii}.{ext}'
        cv2.imwrite(str(dst / 'images' / fixname), s)
    
    #cv2.imwrite(str(dst / 'images' / name), ar)
    
    annf = (ann_path / f.name).with_suffix('.json')
    data = read_ann(annf)
    poly = convert_ann(data, fixer=fix_single_poly)
    mask = rasterize([poly], out_shape=(h,w))
    mask = cv2.resize(mask, (W,H))
    
    ss = splitter(mask, ch, cw)
    #name, ext = name.split('.')
    for ii,  (s,*_) in enumerate(ss):
        fixname = f'{name}_{ii}.{ext}'
        cv2.imwrite(str(dst / 'masks' / fixname), s)
    
    #cv2.imwrite(str(dst / 'masks' / name), mask)
    
    
    # masks.append(mask)
    # imgs.append(ar)
    break

In [ ]:

from collections import defaultdict

from shapely.geometry import Polygon, MultiPolygon

In [ ]:
tdf = df
imgs_path = Path('input/hmib/train_images/')
ann_path = Path('input/hmib/train_annotations/')


In [ ]:
class BadPoly(Exception):pass

def fix_single_poly(d):
    d = np.array(d)
    dd = d.copy()
    p = Polygon(dd)
    try:
        if p.area < 1:
            return None
    except:
        pass

    if not p.is_valid:
        #print(f'{ii} not valid poly')
        broken[row.organ].append(i)
        cnt = 0
        while not p.is_valid:
            d = d[:-1]
            cnt += 1
            p = Polygon(d)#.buffer(0)
            if cnt == 10:
                p = p.buffer(0)
            elif cnt > 10:
                raise Exception

            #elif cnt > 20:

    if isinstance(p, MultiPolygon):
        areas = [i.area for i in p]
        idx = np.argmax(areas)
        p = p[idx]
    return p

In [ ]:
broken = defaultdict(list)
total = defaultdict(int)
polys = defaultdict(list)

for i,row in tqdm(tdf.iterrows(), total=len(tdf)):
    # if i < 2:
    #     continue
    f = imgs_path / str(row['id'])
    f = f.with_suffix('.tiff')
    fd = rasterio.open(f)
    h,w = fd.shape
    fd.close()
    
    annf = (ann_path / f.name).with_suffix('.json')
    data = read_ann(annf)
    #poly = convert_ann(data, buffer=5)
    total[row.organ] += len(data)
    uni = None
    buffer = 0
    
    for ii, d in enumerate(data):
        # if i == 22 and ii == 17:
        #     d = d[:-4]
        # elif i == 23 and ii == 16:
        #     d = d[:-1]
        #     d = d[1:]
        # elif i == 23 and ii == 69:
        #     d = d[:-2]
        #     d = d[1:]
        # elif i == 40 and ii == 6:
        #     d = d[:-2]
        # elif i == 45 and ii == 12:
        #     d = d[:-6]
        # elif i == 45 and ii == 17:
        #     d = d[:-5]
        # elif i == 45 and ii == 55:
        #     d = d[:-4]
        #     d = d[3:]
        # elif i == 0 and ii == 0:
        #     d = d[:-4]
        #     d = d[2:]
        # elif i == 3 and ii == 0:
        #     d = d[:-3]
        # elif i == 7 and ii == 0:
        #     d = d[:-3]
        # elif i == 7 and ii == 3:
        #     d = d[:-8]
        # elif i == 9 and ii == 8:
        #     d = d[:-4]
        # elif i == 9 and ii == 40:
        #     d = d[:-5]
        # elif i == 10 and ii == 4:
        #     d = d[:-7]
        # elif i == 16 and ii == 5:
        #     d = d[:-5]
        # elif i == 20 and ii == 5:
        #     d = d[:-6]
        
        p = fix_single_poly(d)
        if p is None:
            print('bad poly')
            continue
        assert type(p) == Polygon
        assert p.area > 1e-3
        polys[row.organ].append(p)
    
    # mask = rasterize([uni], out_shape=(h,w))
    # break

In [ ]:
i, ii

In [ ]:
# dd = d.copy()
# dd = dd#[:-1]
# dd = dd[0:]

In [ ]:
pp = Polygon(dd)#.buffer(0)

In [ ]:
pp.area > 1e-3

In [ ]:
pp#.buffer(0)

In [ ]:
plt.figure()
plt.plot(d[:,0], d[:,1])
plt.plot(dd[:,0], dd[:,1])

In [ ]:
type(p)

In [ ]:
p.is_valid

In [ ]:
p.area

In [ ]:
plt.figure()
for k,v in polys.items():
    print(k, len(v), total[k], len(v)/total[k])
    pp = polys[k]
    nums = []
    for p in pp:
        nums.append(len(p.exterior.xy[0]))
    nums = np.array(nums)
    print(f'\t {nums.mean(), nums.std()}')
    
    plt.hist(nums.clip(0,800), bins=50, alpha=.5)

In [ ]:
pp

In [ ]:
uni

In [ ]:
rp.buffer(1).area / rp.buffer(0).area

In [ ]:
for p in poly:
    print(p)

In [ ]:
poly